In [ ]:
# PRACTICAL 07: Seasonal Indices using Link Relative Method
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load dataset
path = "Symphony-Data.csv"  # Update with your actual path
df = pd.read_csv(path)

# Step 2: Preprocessing
df = df.drop(columns=['OPEN', 'HIGH', 'LOW', 'VOLUME', 'CHANGE(%)'])
df['DATE'] = pd.to_datetime(df['DATE'], format="%d-%b-%y")
df.sort_values('DATE', inplace=True)

# Step 3: Extract Month and Year
df['Month'] = df['DATE'].dt.month
df['Year'] = df['DATE'].dt.year

# Step 4: Monthly average per year
monthly_avg = df.groupby(['Month', 'Year'])['PRICE'].mean().unstack()

# Step 5: Calculate link relatives (e.g., 2020/2019, 2021/2020, ...)
for i in range(1, len(monthly_avg.columns)):
    prev_year = monthly_avg.columns[i-1]
    curr_year = monthly_avg.columns[i]
    monthly_avg[f'LinkRel_{curr_year}'] = monthly_avg[curr_year] / monthly_avg[prev_year]

# Step 6: Compute average link relatives per month
link_rel_cols = [col for col in monthly_avg.columns if 'LinkRel_' in str(col)]
monthly_avg['Avg_Link_Rel'] = monthly_avg[link_rel_cols].mean(axis=1)

# Step 7: Normalize to make sum = 12 (months)
seasonal_index = monthly_avg['Avg_Link_Rel']
seasonal_index = seasonal_index / seasonal_index.sum() * 12

# Step 8: Deseasonalize original data using seasonal index
for year in monthly_avg.columns[:6]:  # Assuming years 2019 to 2024
    monthly_avg[f'Deseasonalized_{year}'] = monthly_avg[year] / seasonal_index

# Step 9: Plotting
plt.figure(figsize=(14, 10))

# Subplot 1: Original
plt.subplot(3, 1, 1)
for year in monthly_avg.columns[:6]:
    plt.plot(monthly_avg.index, monthly_avg[year], label=str(year), marker='o')
plt.title("Original Monthly Prices")
plt.xlabel("Month")
plt.ylabel("Price")
plt.legend(loc="upper right")
plt.grid()

# Subplot 2: Seasonal Indices
plt.subplot(3, 1, 2)
plt.plot(seasonal_index.index, seasonal_index, marker='o', color='orange', label="Seasonal Index")
plt.title("Seasonal Indices (Link Relative Method)")
plt
